In [ ]:
#pip install python-dateutil

In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.float_format = "{:,.2f}".format
from functools import lru_cache
import datetime
import dateutil.parser as parser
import matplotlib.pyplot as plt

In [2]:
from datetime import datetime

In [3]:
@lru_cache(maxsize=None)
def read_excel(country,afn):
    if country == 'US':
        return pd.read_excel('/Users/oat/Desktop/งาน/data fund/'+country+'/'+afn+'.xlsx',index_col='Morningstar Institutional Category')
    return pd.read_excel('/Users/oat/Desktop/งาน/data fund/'+country+'/'+afn+'.xlsx',index_col='Morningstar Category')

In [4]:
regions = ['Japan']#['China','India','Euro']
afns = ['Flow','NAV','Asset']

In [5]:
@lru_cache(maxsize=None)
def equal_weight_nav(country):
    df = read_excel(country,'NAV').copy()
    j=3
    if country=='Euro':
        j=4
    df2 = pd.DataFrame(columns=['Category']+[i for i in df.columns[j:]],index=df['Group/Investment'])
    df2['Category'] = list(df.index)
    for t in df2.columns[1:]:
        df2[t] = list(df[t])
    df2.drop(['Benchmark 1: S&P 500 TR USD','Benchmark 2: MSCI EAFE PR USD','Peer Group: Display Group','Number of investments ranked','Peer Group Median'],inplace=True)
    
    #equal weight
    for t in df2.columns[1:]:
        ewn = df2[t].copy()
        lenn = len(ewn.dropna())
        if lenn == 0.0:
            lenn = 1
        weight = 1/lenn
        df2[t] = df2[t]*weight

    if isinstance(df2.columns[4],str):
        df2.rename(columns={i:parser.parse(i) for i in df2.columns[1:]},inplace=True)
        
    return df2

In [6]:
#nav ยังไม่ equal
@lru_cache(maxsize=None)
def nav_notequal(country):
    df = read_excel(country,'NAV').copy()
    j=3
    if country in ['Euro','Japan','India']:
        j=4
    df2 = pd.DataFrame(columns=['Category']+[i for i in df.columns[j:]],index=df['Group/Investment'])
    df2['Category'] = list(df.index)
    for t in df2.columns[1:]:
        df2[t] = list(df[t])
    df2.drop(['Benchmark 1: S&P 500 TR USD','Benchmark 2: MSCI EAFE PR USD','Peer Group: Display Group','Number of investments ranked','Peer Group Median'],inplace=True)
    return df2
    if isinstance(df2.columns[4],str):
        df2.rename(columns={i:parser.parse(i) for i in df2.columns[1:]},inplace=True)
    return df2

In [7]:
@lru_cache(maxsize=None)
def top10flow(country):
    df = read_excel(country,'Flow').copy()
    j=3
    if country == 'Euro':
        j=4
    df2 = pd.DataFrame(columns=['Category']+[i for i in df.columns[j:]],index=df['Group/Investment'])
    df2['Category'] = list(df.index)
    for t in df2.columns[1:]:
        df2[t] = list(df[t])
    df2.drop(['Benchmark 1: S&P 500 TR USD','Benchmark 2: MSCI EAFE PR USD','Peer Group: Display Group','Number of investments ranked','Peer Group Median'],inplace=True)
    if isinstance(df2.columns[4],str):
        df2.rename(columns={i:parser.parse(i) for i in df2.columns[1:]},inplace=True)
    
    return df2

In [8]:
def filter_month(a,x,country):
    month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
    df2 = top10flow(country)
    #df_a = df2.filter(regex='2011-01').transpose().iloc[:,:].cumsum()
    df_a = x
    
    for i in month_list:
        df_b = df2.filter(regex=a+'-'+i).transpose().iloc[:,:].cumsum()
        df_a = pd.concat([df_a,df_b])
    
    return df_a

In [9]:
#filter_month('US').to_excel('/Users/oat/Desktop/งาน/data fund/US/test.xlsx')

In [10]:
def filter_d(country):
    df2 = top10flow(country)#.reset_index().set_index(['Group/Investment','Category'])
    df_a = df2.filter(regex='2011-01').transpose().iloc[:,:]
    df_a = filter_month('2011',df_a,country)
    df_b = df2.filter(regex='2012-01').transpose().iloc[:,:]
    df_b = filter_month('2012',df_b,country)
    df_c = df2.filter(regex='2013-01').transpose().iloc[:,:]
    df_c = filter_month('2013',df_c,country)
    df_d = df2.filter(regex='2014-01').transpose().iloc[:,:]
    df_d = filter_month('2014',df_d,country)
    df_e = df2.filter(regex='2015-01').transpose().iloc[:,:]
    df_e = filter_month('2015',df_e,country)
    df_f = df2.filter(regex='2016-01').transpose().iloc[:,:]
    df_f = filter_month('2016',df_f,country)
    df_g = df2.filter(regex='2017-01').transpose().iloc[:,:]
    df_g = filter_month('2017',df_g,country)
    df_h = df2.filter(regex='2018-01').transpose().iloc[:,:]
    df_h = filter_month('2018',df_h,country)
    df_i = df2.filter(regex='2019-01').transpose().iloc[:,:]
    df_i = filter_month('2019',df_i,country)
    df_j = df2.filter(regex='2020-01').transpose().iloc[:,:]
    df_j = filter_month('2020',df_j,country)
    df_k = pd.concat([df_a,df_b,df_c,df_d,df_e,df_f,df_g,df_h,df_i,df_j]).transpose()
    

    return df_k.reset_index().set_index('Group/Investment')

In [11]:
#filter_d('Euro').to_excel('/Users/oat/Desktop/งาน/data fund/Euro/test.xlsx')

In [254]:
def top10nav(country):
    data_weight = nav_notequal(country)[1:].copy()
    df1 = top10flow(country)[1:]
    
    a = set(data_weight.index)
    b = set(df1.index)
    
    con_ind = a&b
    Con2 = a|b
    con_ind = Con2-con_ind
    con_ind = list(con_ind)[1:]
    
    if isinstance(data_weight.columns[4],str):
        data_weight.rename(columns={i:parser.parse(i) for i in data_weight.columns[1:]},inplace=True)
      
    #df2 = top10flow(country).copy()
    
    
    ####### แก้ตรงนี้ต่่อ ให้เป็น cumsum ทีละปี
    df2 = filter_d(country)
    for i in df2.index:
        if i in con_ind:
            df2 = df2.drop(i)
    #df2 = df2[1:]
    #df2_index = df2.index[1:].to_list()
    #df2 = top10flow(country)[top10flow(country).columns[1:]].transpose().cumsum().transpose()
    #######
    
    
    if isinstance(df2.columns[4],str):
        df2.rename(columns={i:parser.parse(i) for i in df2.columns[1:]},inplace=True)
        
    
    columns = list(set(list(data_weight.columns)[1:]) & set(list(df2.columns))) 
    columns.sort()
    
    df = pd.DataFrame(index=list(range(40)),columns=columns) #dfที่มีวันที่เป็นcolumn
    
    #r = df2[df.columns[]].sort_values(by = columns,ascending=False)[:10].index
    
    for t in range(len(df.columns)-2):
        
        name = list(df2.iloc[:,t].sort_values(ascending=False)[:10].index)
        
        while len(name)<10:
            name.append(0.0)
        nav = [data_weight[df.columns[t]][i] for i in name if i!=0.0 and not pd.isna(i)]
        while len(nav)<10:
            nav.append(0.0)
        navtp1 = [data_weight[df.columns[t+1]][i] for i in name if i!=0.0 and not pd.isna(i)]
        while len(navtp1)<10:
            navtp1.append(0.0)
        navtp2 = [data_weight[df.columns[t+2]][i] for i in name if i!=0.0 and not pd.isna(i)]
        while len(navtp2)<10:
            navtp2.append(0.0)
        df[df.columns[t]] = name+nav+navtp1+navtp2
   
        
       
    return df.transpose()
    

In [255]:
start = datetime.now()
start

datetime.datetime(2021, 9, 30, 14, 56, 4, 383537)

In [229]:
top10nav('Japan')

[Timestamp('2011-01-04 00:00:00'),
 Timestamp('2011-01-05 00:00:00'),
 Timestamp('2011-01-06 00:00:00'),
 Timestamp('2011-01-07 00:00:00'),
 Timestamp('2011-01-11 00:00:00'),
 Timestamp('2011-01-12 00:00:00'),
 Timestamp('2011-01-13 00:00:00'),
 Timestamp('2011-01-14 00:00:00'),
 Timestamp('2011-01-17 00:00:00'),
 Timestamp('2011-01-18 00:00:00'),
 Timestamp('2011-01-19 00:00:00'),
 Timestamp('2011-01-20 00:00:00'),
 Timestamp('2011-01-21 00:00:00'),
 Timestamp('2011-01-24 00:00:00'),
 Timestamp('2011-01-25 00:00:00'),
 Timestamp('2011-01-26 00:00:00'),
 Timestamp('2011-01-27 00:00:00'),
 Timestamp('2011-01-28 00:00:00'),
 Timestamp('2011-01-31 00:00:00'),
 Timestamp('2011-02-01 00:00:00'),
 Timestamp('2011-02-02 00:00:00'),
 Timestamp('2011-02-03 00:00:00'),
 Timestamp('2011-02-04 00:00:00'),
 Timestamp('2011-02-07 00:00:00'),
 Timestamp('2011-02-08 00:00:00'),
 Timestamp('2011-02-09 00:00:00'),
 Timestamp('2011-02-10 00:00:00'),
 Timestamp('2011-02-14 00:00:00'),
 Timestamp('2011-02-

In [ ]:
top10nav('Japan')

In [ ]:
end = datetime.now()
end

In [26]:
(end - start)/60

datetime.timedelta(seconds=13, microseconds=93916)

In [288]:
top10nav('Japan').to_excel('/Users/oat/Desktop/งาน/data fund/Japan/Japan_month.xlsx')

In [93]:
for country in regions:
    top10nav(country).to_excel('/Users/oat/Desktop/งาน/data fund/'+country+'/'+country+'_month.xlsx')
    print(country)

KeyboardInterrupt: 